## Calculating weighted Beta of Portfolio using a benchmark


In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats
from pandas_datareader import data as pdr

### Set date range 

In [2]:
start = dt.datetime(2022,1,1) #YTD
end = dt.datetime.now()
start, end

(datetime.datetime(2022, 1, 1, 0, 0),
 datetime.datetime(2022, 8, 15, 17, 52, 11, 283775))

###  Make list of your stocks and benchmark

In [3]:
stocks = ['AAPL', 'AMZN', 'GOOG', 'NFLX', 'TSLA', 'MSFT', 'NVDA', 'JPM', 'COIN']
tickers = ['^IXIC', 'AAPL', 'AMZN', 'GOOG', 'NFLX', 'TSLA', 'MSFT', 'NVDA', 'JPM', 'COIN']

### Calculate daily price changes

In [4]:
df = pdr.get_data_yahoo(tickers, start, end)
log_returns = np.log(df.Close / df.Close.shift(1)).dropna()
log_returns.head()

Symbols,^IXIC,AAPL,AMZN,GOOG,NFLX,TSLA,MSFT,NVDA,JPM,COIN
Date,,,,,,,,,,
2022-01-04,-0.013357,-0.012773,-0.017060,-0.004546,-0.010467,-0.042733,-0.017296,-0.027976,0.037209,-0.003591
2022-01-05,-0.034020,-0.026960,-0.019074,-0.047962,-0.040794,-0.054954,-0.039144,-0.059285,-0.024428,-0.065757
2022-01-06,-0.001280,-0.016834,-0.006734,-0.000745,-0.025394,-0.021758,-0.007933,0.020581,0.010568,-0.000982
2022-01-07,-0.009659,0.000988,-0.004297,-0.003981,-0.022352,-0.036090,0.000510,-0.033598,0.009859,-0.007162
2022-01-10,0.000464,0.000116,-0.006592,0.011391,-0.002239,0.029891,0.000732,0.005600,0.000957,-0.032014


### Calculate beta for each stock

Covariance(Benchmark, Stock) / Variance(Benchmark)

In [5]:
def calculate_beta(df):
    np_array = df.values
    b = np_array[:,0]
    beta = []
    for ind, col in enumerate(df):
        if ind > 0:
            s = np_array[:,ind]
            covariance = np.cov(s,b)
            beta.append(covariance[0,1]/covariance[1,1])
            
    return pd.Series(beta, df.columns[1:], name = 'Beta')

In [6]:
calculate_beta(log_returns)

Symbols
AAPL    0.934180
AMZN    1.336169
GOOG    1.011038
NFLX    1.480720
TSLA    1.653009
MSFT    0.934369
NVDA    1.787397
JPM     0.547392
COIN    2.650938
Name: Beta, dtype: float64

In [7]:
beta = calculate_beta(log_returns)

### Calculate Weighted Beta of Portfolio

In [8]:
units = np.array([10, 10, 10, 10, 3, 10, 10, 15, 20])
ASXprices = df.Close[-1:].values.tolist()[0]
price = np.array([round(price,2) for price in ASXprices[1:]])
value = units*price
weight = [round(val/sum(value),2) for val in value]
beta = round(beta, 2)

In [9]:
portfolio = pd.DataFrame({
    'Stock': stocks,
    'Price': price,
    'Shares amount': units,
    'Market Value': value,
    'Weight': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
portfolio

,Stock,Price,Shares amount,Market Value,Weight,Beta,Weighted Beta
Symbols,,,,,,,
AAPL,AAPL,171.98,10,1719.80,0.10,0.93,0.0930
AMZN,AMZN,141.78,10,1417.80,0.08,1.34,0.1072
GOOG,GOOG,121.74,10,1217.40,0.07,1.01,0.0707
NFLX,NFLX,249.64,10,2496.40,0.14,1.48,0.2072
TSLA,TSLA,913.26,3,2739.78,0.15,1.65,0.2475
MSFT,MSFT,290.48,10,2904.80,0.16,0.93,0.1488
NVDA,NVDA,186.81,10,1868.10,0.10,1.79,0.1790
JPM,JPM,121.22,15,1818.30,0.10,0.55,0.0550
COIN,COIN,90.52,20,1810.40,0.10,2.65,0.2650


#### Weighted Beta of Portfolio

In [10]:
sum(weight*beta)

1.3734000000000002